# Chapter 4: Training  Models


## Linear Regression

a linear model makes a predictipon by simply computing a weighted sum of the input features, plus a constant called the bias term:

$$\hat{y}=\theta_{0}+\theta_{1} x_{1}+\theta_{2} x_{2}+\cdots+\theta_{n} x_{n}$$

$$\hat{y}=h_{\theta}(\mathbf{x})=\boldsymbol{\theta} \cdot \mathbf{x}$$

### Normal Equation

$$\widehat{\boldsymbol{\theta}}=\left(\mathbf{X}^{\top} \mathbf{X}\right)^{-1} \mathbf{X}^{\top} \mathbf{y}$$

- This equation gives us the weights directly
- pseudo inverse can be used instead of inverse, as $\left(\mathbf{X}^{\top} \mathbf{X}\right)$ might be singular, as result of redundant features or not enough instances


scikit learn LinearRegression class uses normal equations and is based on scipy.linalg.lstsq() which computes pseudo inverse using SVD.

computing inverse of this matrix has a computational complexity of $O(n^2.4)$ to $O(n^3)$ depending on the implementation. scikit learn approach's complexity is about $O(n^2)$.

normal equation and svd method get very slow as numver of features ($n$) grows. On the positive side they are linear regarding to the number of instances.

## Gradient Descent

Gradient Descent algorithm tweaks parameters iteratively in order to minimize a cost function.

### Batch Gradient Descent

batch gradient descent computes the partial derivative if the cost function with regard to parameter $\theta_j$ :
$$\frac{\partial}{\partial \theta_{j}} \operatorname{MSE}(\boldsymbol{\theta})=\frac{2}{m} \sum_{i=1}^{m}\left(\boldsymbol{\theta}^{\top} \mathbf{x}^{(i)}-y^{(i)}\right) x_{j}^{(i)}$$

We can re write this in the form below to compute all the derivatives in one go.The gradient vector contains all the derivatives of the cost function:
$$\nabla_{\theta} \operatorname{MSE}(\boldsymbol{\theta})=\left(\begin{array}{c}
\frac{\partial}{\partial \theta_{0}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\frac{\partial}{\partial \theta_{1}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\vdots \\
\frac{\partial}{\partial \theta_{\mathrm{n}}} \operatorname{MSE}(\boldsymbol{\theta})
\end{array}\right)=\frac{2}{m} \mathbf{X}^{\top}(\mathbf{X} \boldsymbol{\theta}-\mathbf{y})$$

- This method computes the derivatives over the whole training set $\mathbf{X}$ at each step. this is why it's called Batch Gradient Descent. As a result it gets slow on very large training sets. However, Gradient Descent scales well with the number of features.

Once we have the gradient vector, we can calculate new weights:
$$\boldsymbol{\theta}^{(\text {next step })}=\boldsymbol{\theta}-\eta \nabla_{\boldsymbol{\theta}} \operatorname{MSE}(\boldsymbol{\theta})$$ where $\eta$ is the learning rate

### Stochastic Gradient Descent

Stochastic Gradient Descent, unlike Batch Gradient Descent, picks a random instance in training set at every training step and computes gradients based on that single instance. Over time it will get close to the minimum, but once it gets there it will continue to bounce around, never settling down. So once the algorithm stops, the final parameter values are good, but not optimal.

Pros:
- faster convergence
- more likely to find global minimum
- can be implemented as an out-of-core algorithm

While the randomness is good for escaping from local optima by jumping over, it's bad as it can never settle at the minimum. one solution to this problem is to gradually reduce the learning rate. the learning rate should not be reduced too quickly or too slowly. The function that determines the learning rate at each iteration is called the learning schedule.

In SGD implementation, epochs are used instead of iterations. While BGD calculates over the whole data set in every iteration, SGD calculates derivatives over every instance separately in every epoch; so it goes through the training set on each epoch.

When using SGD, the training instances must be independent and identically disturbed to ensure that the parameters get pulled toward the global optimum. A simple way to ensure this is to shuffle the instances
during training (e.g., pick each instance randomly, or shuffle the training set at the
beginning of each epoch). If you do not shuffle the instances—for example, if the
instances are sorted by label—then SGD will start by optimizing for one label, then the
next, and so on, and it will not settle close to the global minimum.


Scikit-Learn has implemented SGD in SGDRegressor class.

### Mini-Batch Gradient Descent

Mini-Batch GD, instead of training on whole training set as in BGD or training on one instance as in SGD on each step, computes gradients on a small random set of instances. Mini-Batch GD benefits from hardware optimization of matrix operations, especially when using GPUs. It will end up walking around a bit closer to the minimum than
Stochastic GD—but it may be harder for it to escape from local minima.

$$\begin{array}{llllll}
\hline \text { Algorithm } & \begin{array}{l}
\text { Large } \\
m
\end{array} & \begin{array}{l}
\text { Out-of-core } \\
\text { support }
\end{array} & \begin{array}{l}
\text { Large } \\
\boldsymbol{n}
\end{array} & \text { Hyperparams } & \begin{array}{l}
\text { Scaling } \\
\text { required }
\end{array} & \text { Scikit-Learn } \\
\hline \begin{array}{l}
\text { Normal } \\
\text { Equation }
\end{array} & \text { Fast } & \text { No } & \text { Slow } & 0 & \text { No } & \text { N/A } \\
\hline \text { SVD } & \text { Fast } & \text { No } & \text { Slow } & 0 & \text { No } & \text { LinearRegression } \\
\hline \text { Batch GD } & \text { Slow } & \text { No } & \text { Fast } & 2 & \text { Yes } & \text { SGDRegressor } \\
\hline \begin{array}{l}
\text { Stochastic } \\
\text { GD }
\end{array} & \text { Fast } & \text { Yes } & \text { Fast } & \geq 2 & \text { Yes } & \text { SGDRegressor } \\
\hline \begin{array}{l}
\text { Mini-batch } \\
\text { GD }
\end{array} & \text { Fast } & \text { Yes } & \text { Fast } & \geq 2 & \text { Yes } & \text { SGDRegressor }
\end{array}$$

## Polynomial Learning

We can use linear regression models to train non linear data by adding combinations and higher degrees of features. This can be done with scikit-learn PloynomialFeatures class from <c>preprocessing</c> package. Note that when there are multiple features, Polynomial Regression is capable
of finding relationships between features (which is something a plain Linear
Regression model cannot do). This is made possible by the fact that
PolynomialFeatures also adds all combinations of features up to the given
degree. 

## Learning Curves

One way to see if a model is over-fitting or not was to use cross-validation metrics. Another way to tell is to look at the learning curves of a model; these are plots of model performance on the training set and validation set as function of the training set size or the training iteration (checkout training_models notebook for examples).
generally the train error increases as size of the training set grows, but validation error decreases. Eventually these two get close to each other and end up in plateau and don't change as the training set gets bigger. If both curves reach a plateau close to each other and relatively high; the model is probably under-fitting. On the other hand if plateaus are far from each other (low train error and high validation error) the model is over-fitting.


### Bias-Variance Trade-off

A model's generalization error can be expressed as sum of three errors:
- #### Bias
this part of error is due to wrong assumptions, such as assuming that data is linear when it's actually quadratic. A high-bias model is likely to under-fit the training data.
- #### Variance
This part is due to the model’s excessive sensitivity to small
variations in the training data. A model with many degrees of
freedom (such as a high-degree polynomial model) is likely to have
high variance and thus overfit the training data.
- #### Irreducible error
This part is due to the noisiness of the data itself.

Increasing a model’s complexity will typically increase its variance and reduce its bias. Conversely, reducing a model’s complexity increases its bias and reduces its variance. This is why it is called a trade-off.


## Regularized Linear Models

### Ridge Regression

Ridge regression is a regularized version of linear regression which a regularization term equal to $\alpha \sum_{i=1}^{n} \theta_{i}^{2}$ is added to cost function. The hyperparameter $\alpha$ controls how much you want to regularize the model.
$$J(\boldsymbol{\theta})=\operatorname{MSE}(\boldsymbol{\theta})+\alpha \frac{1}{2} \sum_{i=1}^{n} \theta_{i}^{2}$$
Note that bias therm $\theta_0$ is not regularized. 

If we define $\mathbf{w}$ as the vector of feature weights (θ1 to θn), then the
regularization term is equal to $1 / 2\left(\|\mathbf{w}\|_{2}\right)^{2},$ where $\|\mathbf{w}\|_{2}$represents the $\ell_{2}$ norm of the weight vector. For Gradient Descent, just add $\alpha\mathbf{w}$ to the $\operatorname{MSE}$ gradient vector.


- #### WARNING
It is important to scale the data before performing Ridge Regression, as it is sensitive to the scale of the input features.

Ridge Regression can be implemented in both closed form using a matrix factorization technique (Cholesky) and Gradient Descent.<br>
Closed Form method can show in the following form in which $\mathbf{A}$ is the $(n+1)×(n+1)$ identity matrix, except with a 0 in the top-left cell. corresponding to the bias term:
$$\widehat{\boldsymbol{\theta}}=\left(\mathbf{X}^{\top} \mathbf{X}+\alpha \mathbf{A}\right)^{-1} \mathbf{X}^{\top} \mathbf{y}$$
Ridge Regression can be performed with scikit-learn Ridge class in linear_models package with solver="cholesky" parameter; and using SGD with penalty="l2" parameter.